In [224]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import datetime
import gc
import gc
import os
import random
import xgboost as xgb
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.cluster import FeatureAgglomeration

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge

from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle

In [0]:
train=pd.read_csv('/content/drive/My Drive/Train (7).csv')
test=pd.read_csv('/content/drive/My Drive/Test (4).csv')
sub=pd.read_csv('/content/drive/My Drive/SampleSubmission (8).csv')

Shifting **Features** 

In [0]:



train['precipitable_water_entire_atmosphere123'] = train.temperature_2m_above_ground.shift(periods=2)
test['precipitable_water_entire_atmosphere123'] = test.temperature_2m_above_ground.shift(periods=2)
train['precipitable_water_entire_atmosphere12'] = train.temperature_2m_above_ground.shift(periods=3)
test['precipitable_water_entire_atmosphere12'] = test.temperature_2m_above_ground.shift(periods=3)
#temperature_2m_above_ground	u_component_of_wind_10m_above_ground
train['precipitable_water_entire_atmosphere av hier'] = train.precipitable_water_entire_atmosphere.shift(periods=2)
train['precipitable_water_entire_atmosphere av av hier'] = train.precipitable_water_entire_atmosphere.shift(periods=3)
train['precipitable_water_entire_atmosphere av av av hier'] = train.precipitable_water_entire_atmosphere.shift(periods=4)

test['precipitable_water_entire_atmosphere av hier'] = test.precipitable_water_entire_atmosphere.shift(periods=2)
test['precipitable_water_entire_atmosphere av av hier'] = test.precipitable_water_entire_atmosphere.shift(periods=3)

test['precipitable_water_entire_atmosphere av av av hier'] = test.precipitable_water_entire_atmosphere.shift(periods=4)

In [0]:
train['L3_NO2_NO2_column_number_density2']=train['L3_NO2_NO2_column_number_density'].shift(periods=2)
train['L3_O3_O3_column_number_density2']=train['L3_O3_O3_column_number_density'].shift(periods=2)
train['L3_HCHO_tropospheric_HCHO_column_number_density2']=train['L3_HCHO_tropospheric_HCHO_column_number_density'].shift(periods=2)
train['L3_CO_CO_column_number_density2']=train['L3_CO_CO_column_number_density'].shift(periods=2)
train['L3_CLOUD_surface_albedo2']=train['L3_CLOUD_surface_albedo'].shift(periods=2)
train['L3_CLOUD_cloud_optical_depth2']=train['L3_CLOUD_cloud_optical_depth'].shift(periods=2)

In [0]:
#using data of yesterday 

In [0]:
l = [np.NaN]
s=train['precipitable_water_entire_atmosphere'][0:len(train)-1]
for i in s :
  l.append(i)
train['pr hier']=l
l = [np.NaN]
s=test['precipitable_water_entire_atmosphere'][0:len(test)-1]
for i in s :
  l.append(i)
test['pr hier']=l

In [0]:
l = [np.NaN]
s=train['temperature_2m_above_ground'][0:len(train)-1]
for i in s :
  l.append(i)
train['temp hier']=l
l = [np.NaN]
s=test['temperature_2m_above_ground'][0:len(test)-1]
for i in s :
  l.append(i)
test['temp hier']=l

In [232]:
for i in range (len(test)):
  if i%94==0 :
    test['pr hier'][i]=np.NaN
for i in range (len(train)):
  if i%94==0 :
    train['pr hier'][i]=np.NaN

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [233]:
train.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,...,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,precipitable_water_entire_atmosphere123,precipitable_water_entire_atmosphere12,precipitable_water_entire_atmosphere av hier,precipitable_water_entire_atmosphere av av hier,precipitable_water_entire_atmosphere av av av hier,L3_NO2_NO2_column_number_density2,L3_O3_O3_column_number_density2,L3_HCHO_tropospheric_HCHO_column_number_density2,L3_CO_CO_column_number_density2,L3_CLOUD_surface_albedo2,L3_CLOUD_cloud_optical_depth2,pr hier,temp hier
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,...,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,61.402626,-74.457583,...,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.000050,-1.452612,0.059433,-14.708036,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.000000,18.516840
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,0.00750,27.031030,5.065727,3.500559,0.000067,0.000170,-1.001242,0.015904,841410.713456,-105.201338,49.839714,-78.342701,34.296977,0.000059,7311.869141,0.000007,0.115

In [234]:
for i in range (len(test)):
  if i%94==0 :
    test['temp hier'][i]=np.NaN
for i in range (len(train)):
  if i%94==0 :
    train['temp hier'][i]=np.NaN

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [235]:
l = [np.NaN]
s=train['relative_humidity_2m_above_ground'][0:len(train)-1]
for i in s :
  l.append(i)
train['HUMID hier']=l
l = [np.NaN]
s=test['relative_humidity_2m_above_ground'][0:len(test)-1]
for i in s :
  l.append(i)
test['HUMID hier']=l
for i in range (len(test)):
  if i%94==0 :
    test['HUMID hier'][i]=np.NaN
for i in range (len(train)):
  if i%94==0 :
    train['HUMID hier'][i]=np.NaN

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Drop columns which are not importants

In [0]:
test=test.drop(columns=[
 
 'L3_HCHO_sensor_azimuth_angle',
 'L3_HCHO_sensor_zenith_angle',
 'L3_HCHO_solar_azimuth_angle',
 'L3_HCHO_solar_zenith_angle',
])
train=train.drop(columns=[
                          
 
 'L3_HCHO_sensor_azimuth_angle',
 'L3_HCHO_sensor_zenith_angle',
 'L3_HCHO_solar_azimuth_angle',
 'L3_HCHO_solar_zenith_angle',
 ])
test=test.drop(columns=[
                        #'L3_CO_sensor_azimuth_angle',
 'L3_CO_sensor_zenith_angle',
 #'L3_CO_solar_azimuth_angle',
 'L3_CO_solar_zenith_angle',
 'L3_SO2_sensor_azimuth_angle',
 'L3_SO2_sensor_zenith_angle',
 'L3_SO2_solar_azimuth_angle',
 'L3_SO2_solar_zenith_angle']

)
train=train.drop(columns=[
                          #'L3_CO_sensor_azimuth_angle',
 'L3_CO_sensor_zenith_angle',
 #'L3_CO_solar_azimuth_angle',
 'L3_CO_solar_zenith_angle',
 

 'L3_SO2_sensor_azimuth_angle',
 'L3_SO2_sensor_zenith_angle',
 'L3_SO2_solar_azimuth_angle',
 'L3_SO2_solar_zenith_angle']                                

 )

In [0]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table


In [0]:
p=[]
l=list(test.columns)
l.append('target')
for c in list(train.columns):
  if c not in l:
    p.append(c)
    train.drop(c,inplace=True,axis=1)

In [239]:
[col for col in train.columns if not train[col].isnull().any()]

['Place_ID X Date',
 'Date',
 'Place_ID',
 'target',
 'precipitable_water_entire_atmosphere',
 'relative_humidity_2m_above_ground',
 'specific_humidity_2m_above_ground',
 'temperature_2m_above_ground',
 'u_component_of_wind_10m_above_ground',
 'v_component_of_wind_10m_above_ground']

In [240]:
result_train=missing_zero_values_table(train)

Your selected dataframe has 76 columns and 30557 Rows.
There are 66 columns that have missing values.


In [0]:
k=['L3_CH4_solar_zenith_angle','L3_CH4_solar_azimuth_angle','L3_CH4_sensor_zenith_angle','L3_CH4_sensor_azimuth_angle','L3_CH4_aerosol_optical_depth','L3_CH4_aerosol_height','L3_CH4_CH4_column_volume_mixing_ratio_dry_air']
train.drop(k,inplace=True,axis=1)
test.drop(k,inplace=True,axis=1)

In [0]:

h=['L3_NO2_NO2_column_number_density', 	'L3_NO2_NO2_slant_column_number_density', 	'L3_NO2_absorbing_aerosol_index', 	'L3_NO2_cloud_fraction', 	'L3_NO2_sensor_altitude', 	'L3_NO2_sensor_azimuth_angle', 	'L3_NO2_sensor_zenith_angle', 	'L3_NO2_solar_azimuth_angle' 	,'L3_NO2_solar_zenith_angle', 	'L3_NO2_stratospheric_NO2_column_number_density', 	'L3_NO2_tropopause_pressure', 	'L3_NO2_tropospheric_NO2_column_number_density']

In [0]:


r=['L3_CLOUD_cloud_base_height', 	'L3_CLOUD_cloud_base_pressure' ,	'L3_CLOUD_cloud_fraction', 	'L3_CLOUD_cloud_optical_depth', 	'L3_CLOUD_cloud_top_height', 	'L3_CLOUD_cloud_top_pressure', 	'L3_CLOUD_sensor_azimuth_angle', 	'L3_CLOUD_sensor_zenith_angle', 	'L3_CLOUD_solar_azimuth_angle', 	'L3_CLOUD_solar_zenith_angle', 	'L3_CLOUD_surface_albedo']

In [0]:
p=['L3_AER_AI_absorbing_aerosol_index' ,	'L3_AER_AI_sensor_altitude', 	'L3_AER_AI_sensor_azimuth_angle' ,	'L3_AER_AI_sensor_zenith_angle', 	'L3_AER_AI_solar_azimuth_angle' 	,'L3_AER_AI_solar_zenith_angle']

**Extracting** **Informations** ***from*** **date**

In [245]:
train["Date"] = pd.to_datetime(train['Date'])
test["Date"] = pd.to_datetime(test['Date'])

def applyer(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0 






def makedate(df):
      
   df['day of week']=df['Date'].dt.dayofweek
   df['weekend'] = df['Date'].apply(applyer)
   df['month']=0
   df['day']=0
   df['winter']=0
   for i in range(len(df)):  
           
           df['month'][i]=df['Date'][i].month 
           df['day'][i]=df['Date'][i].day
           if (df['month'][i]==1 or df['month'][i]==2 ):
             df['winter'][i]=1

makedate(train)
makedate(test)


#test["day"] = test.Date.dt.day
#train["day"] = train.Date.dt.day

#train["month"] = train.Date.dt.month

#test["month"] = test.Date.dt.month

#train["day"] = train.Date.dt.dayofweek

#test["day"] = test.Date.dt.dayofweek

train["quarter"] = train.Date.dt.quarter

test["quarter"] = test.Date.dt.quarter


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Fillna

In [0]:
train=train.fillna(method='ffill')
train=train.fillna(train.mean())
test=test.fillna(method ='ffill')
test=test.fillna(test.mean())

In [247]:
air_temperature_filler = pd.DataFrame(train.groupby(['Place_ID','month'])['temperature_2m_above_ground'].mean())
train['mean temp per month']=0
for i in range (len(train)):
  
  train['mean temp per month'][i]=air_temperature_filler.loc[(train['Place_ID'][i], train['month'][i]), :]

air_temperature_filler = pd.DataFrame(test.groupby(['Place_ID','month'])['temperature_2m_above_ground'].mean())

test['mean temp per month']=0
for i in range (len(test)):
  
  test['mean temp per month'][i]=air_temperature_filler.loc[(test['Place_ID'][i], test['month'][i]), :]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


# PCA

In [0]:
pca = PCA(random_state=42,n_components=1)
pg_features =  train.filter(regex='L3_AER_.*')
train_pca = pca.fit_transform(pg_features)
train['pca_AE'] = train_pca[:,0]
pg_features =  test.filter(regex='L3_AER.*')
test_pca = pca.transform(pg_features)
test['pca_AE'] = test_pca[:,0]
pca = PCA(random_state=42,n_components=1)
pg_features =  train.filter(regex='L3_NO2_.*')
train_pca = pca.fit_transform(pg_features)
train['pca_NO'] = train_pca[:,0]
pg_features =  test.filter(regex='L3_NO2.*')
test_pca = pca.transform(pg_features)
test['pca_NO'] = test_pca[:,0]


In [0]:
trainag=train.copy()
testag=test.copy()

In [0]:
train.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
test.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
target=train['target']
train.drop('target',axis=1,inplace=True)

In [0]:
feature=['precipitable_water_entire_atmosphere', 	'relative_humidity_2m_above_ground', 	'specific_humidity_2m_above_ground', 	'temperature_2m_above_ground', 	'u_component_of_wind_10m_above_ground', 	'v_component_of_wind_10m_above_ground', 	'L3_NO2_NO2_column_number_density', 'L3_SO2_SO2_column_number_density',	'L3_SO2_SO2_column_number_density_amf' 	,'L3_SO2_SO2_slant_column_number_density', 	 'L3_NO2_tropospheric_NO2_column_number_density', 'L3_HCHO_tropospheric_HCHO_column_number_density', 'L3_CO_CO_column_number_density', 'L3_HCHO_HCHO_slant_column_number_density'	,'L3_CO_H2O_column_number_density'	,'L3_O3_O3_column_number_density'	,'L3_NO2_NO2_slant_column_number_density']
train1=train.copy()
test1=test.copy()
for c in list(feature):
  if c not in l:
    train1.drop(c,inplace=True,axis=1)
    test1.drop(c,inplace=True,axis=1)
    
pca = PCA(random_state=42,n_components=1)
train_pca = pca.fit_transform(train1)
train['pca_feature'] = train_pca[:,0]
test_pca = pca.transform(test1)
test['pca_feature'] = test_pca[:,0]

In [0]:

from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
dev_X, val_X, dev_y, val_y = train_test_split(train, target, test_size = 0.15, random_state = 42)


In [0]:
#FIRST MODEL

In [253]:
#without agg
from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=30000,
                             learning_rate=0.045,
                             depth=8,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=300)
cb_model.fit(dev_X, dev_y,
             eval_set=(val_X, val_y),
             use_best_model=True,
             verbose=50)

0:	learn: 46.2279628	test: 45.4987457	best: 45.4987457 (0)	total: 85.3ms	remaining: 42m 38s
50:	learn: 33.1329302	test: 32.6519092	best: 32.6519092 (50)	total: 3.8s	remaining: 37m 9s
100:	learn: 30.3358613	test: 30.4029859	best: 30.4029859 (100)	total: 7.43s	remaining: 36m 40s
150:	learn: 28.6353202	test: 29.3068574	best: 29.3068574 (150)	total: 11.1s	remaining: 36m 40s
200:	learn: 27.3830742	test: 28.6629251	best: 28.6629251 (200)	total: 14.8s	remaining: 36m 32s
250:	learn: 26.2197157	test: 28.0718998	best: 28.0718998 (250)	total: 18.4s	remaining: 36m 20s
300:	learn: 25.1718557	test: 27.5588216	best: 27.5588216 (300)	total: 22s	remaining: 36m 7s
350:	learn: 24.2623447	test: 27.2137759	best: 27.2137759 (350)	total: 25.6s	remaining: 36m
400:	learn: 23.4475404	test: 26.8827536	best: 26.8827536 (400)	total: 29.2s	remaining: 35m 53s
450:	learn: 22.7152424	test: 26.6210670	best: 26.6210670 (450)	total: 32.8s	remaining: 35m 49s
500:	learn: 22.0083652	test: 26.3468329	best: 26.3468329 (500)	t

In [0]:
pred_test_cat2 = (cb_model.predict(test))


In [0]:
#train['L3_SO2_SO2_column_number_density1'] = train.L3_SO2_SO2_column_number_density.shift(periods=1)
#test['L3_SO2_SO2_column_number_density1'] = train.L3_SO2_SO2_column_number_density.shift(periods=1)

#train['L3_NO2_tropospheric_NO2_column_number_density1'] = train.specific_humidity_2m_above_ground.shift(periods=1)
#test['L3_NO2_tropospheric_NO2_column_number_density1'] = test.specific_humidity_2m_above_ground.shift(periods=1)

In [0]:
#dont run this xD
#train['relative_humidity_2m_above_ground1'] = train.relative_humidity_2m_above_ground.shift(periods=1)
#test['relative_humidity_2m_above_ground1'] = train.relative_humidity_2m_above_ground.shift(periods=1)

#train['specific_humidity_2m_above_ground1'] = train.specific_humidity_2m_above_ground.shift(periods=1)
#test['specific_humidity_2m_above_ground1'] = test.specific_humidity_2m_above_ground.shift(periods=1)

#train['temperature_2m_above_ground1'] = train.temperature_2m_above_ground.shift(periods=1)
#test['temperature_2m_above_ground1'] = train.temperature_2m_above_ground.shift(periods=1)

#train['u_component_of_wind_10m_above_ground1'] = train.u_component_of_wind_10m_above_ground.shift(periods=1)
#test['u_component_of_wind_10m_above_ground1'] = test.u_component_of_wind_10m_above_ground.shift(periods=1)

#train['v_component_of_wind_10m_above_ground1'] = train.v_component_of_wind_10m_above_ground.shift(periods=1)
#test['v_component_of_wind_10m_above_ground1'] = train.v_component_of_wind_10m_above_ground.shift(periods=1)

#train['precipitable_water_entire_atmosphere1'] = train.precipitable_water_entire_atmosphere.shift(periods=1)
#test['precipitable_water_entire_atmosphere1'] = test.precipitable_water_entire_atmosphere.shift(periods=1)

In [257]:
trainag.head()

,Place_ID X Date,Date,Place_ID,target,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_solar_azimuth_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,precipitable_water_entire_atmosphere123,precipitable_water_entire_atmosphere12,precipitable_water_entire_atmosphere av hier,precipitable_water_entire_atmosphere av av hier,precipitable_water_entire_atmosphere av av av hier,pr hier,temp hier,HUMID hier,day of week,weekend,month,day,winter,quarter,mean temp per month,pca_AE,pca_NO
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,-61.789016,-0.000010,0.000000,0.000064,0.566828,2505.218548,77542.072128,0.000000,17.950029,3280.496698,70847.513552,76.536426,38.593017,-61.752587,22.363665,0.226013,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,9.321311,9.321471,15.302323,15.302591,15.302886,15.305183,9.323654,70.551504,3,0,1,2,1,1,22,7658.699707,-83286.588393
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,-74.457583,0.000114,0.059433,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.000050,-1.452612,0.059433,9.321311,9.321471,15.302323,15.302591,15.302886,11.000000,18.516840,60.200001,4,0,1,3,1,1,22,8221.641733,-83849.569586
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,16.400000,33.400002,0.00750,27.031030,5.065727,3.500559,0.000067,0.000170,-1.001242,0.015904,841410.713456,-105.201338,49.839714,-78.342701,34.296977,0.000059,7311.869141,0.000007,0.115876,232.233484,0.082063,-105.201338,49.839714,-78.342701,34.296977,0.020677,1109.347101,134.700335,841319.860448,-103.494458,-78.355069,0.000027,0.082063,0.000124,0.

 #using agg

In [0]:
aggs = {}
aggs['L3_HCHO_tropospheric_HCHO_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_CO_CO_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_O3_O3_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_NO2_NO2_slant_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_NO2_NO2_column_number_density'] = ['sum','max','min','mean','std']

aggs['L3_O3_cloud_fraction'] = ['max','min','mean','std']
aggs['L3_CO_H2O_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_HCHO_HCHO_slant_column_number_density'] = ['sum','max','min','mean','std']


aggs['relative_humidity_2m_above_ground'] = ['sum','max','min','mean','std']
aggs['u_component_of_wind_10m_above_ground'] = ['sum','max','min','mean','std']


aggs['precipitable_water_entire_atmosphere'] = ['sum','max','min','mean','std']
aggs['specific_humidity_2m_above_ground'] = ['sum','max','min','mean','std']

aggs['v_component_of_wind_10m_above_ground'] = ['sum','max','min','mean','std']
aggs['temperature_2m_above_ground'] = ['sum','max','min','mean','std']


aggs['pca_AE'] = ['sum','max','min','mean','std']


In [259]:
agg_trans = trainag.groupby(['Place_ID']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (trainag.groupby('Place_ID')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

agg_trans = pd.merge(df, agg_trans, on='Place_ID', how='left')
trainag = pd.merge(trainag,agg_trans, on='Place_ID', how='left')
trainag.head()

,Place_ID X Date,Date,Place_ID,target,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_solar_azimuth_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,...,L3_HCHO_HCHO_slant_column_number_density_sum,L3_HCHO_HCHO_slant_column_number_density_max,L3_HCHO_HCHO_slant_column_number_density_min,L3_HCHO_HCHO_slant_column_number_density_mean,L3_HCHO_HCHO_slant_column_number_density_std,relative_humidity_2m_above_ground_sum,relative_humidity_2m_above_ground_max,relative_humidity_2m_above_ground_min,relative_humidity_2m_above_ground_mean,relative_humidity_2m_above_ground_std,u_component_of_wind_10m_above_ground_sum,u_component_of_wind_10m_above_ground_max,u_component_of_wind_10m_above_ground_min,u_component_of_wind_10m_above_ground_mean,u_component_of_wind_10m_above_ground_std,precipitable_water_entire_atmosphere_sum,precipitable_water_entire_atmosphere_max,precipitable_water_entire_atmosphere_min,precipitable_water_entire_atmosphere_mean,precipitable_water_entire_atmosphere_std,specific_humidity_2m_above_ground_sum,specific_humidity_2m_above_ground_max,specific_humidity_2m_above_ground_min,specific_humidity_2m_above_ground_mean,specific_humidity_2m_above_ground_std,v_component_of_wind_10m_above_ground_sum,v_component_of_wind_10m_above_ground_max,v_component_of_wind_10m_above_ground_min,v_component_of_wind_10m_above_ground_mean,v_component_of_wind_10m_above_ground_std,temperature_2m_above_ground_sum,temperature_2m_above_ground_max,temperature_2m_above_ground_min,temperature_2m_above_ground_mean,temperature_2m_above_ground_std,pca_AE_sum,pca_AE_max,pca_AE_min,pca_AE_mean,pca_AE_std
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,-61.789016,-0.000010,0.000000,0.000064,0.566828,2505.218548,...,0.00374,0.000151,-0.000152,0.00004,0.000048,3860.300067,84.5,13.5,41.067022,17.631103,232.749067,5.065727,0.58197,2.476054,0.751495,1341.80793,30.300001,5.5,14.274552,5.13326,0.562116,0.01016,0.00236,0.00598,0.001949,87.158944,6.560994,-2.20218,0.927223,1.743238,1925.474463,27.980219,11.452844,20.483771,3.414758,766090.842377,8859.225542,7427.448255,8149.902578,371.611739
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,-74.457583,0.000114,0.059433,0.000171,0.858446,175.019862,...,0.00374,0.000151,-0.000152,0.00004,0.000048,3860.300067,84.5,13.5,41.067022,17.631103,232.749067,5.065727,0.58197,2.476054,0.751495,1341.80793,30.300001,5.5,14.274552,5.13326,0.562116,0.01016

In [260]:
agg_trans = testag.groupby(['Place_ID']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (testag.groupby('Place_ID')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

agg_trans = pd.merge(df, agg_trans, on='Place_ID', how='left')
testag = pd.merge(testag,agg_trans, on='Place_ID', how='left')
testag.head()

,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_solar_azimuth_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,...,L3_HCHO_HCHO_slant_column_number_density_sum,L3_HCHO_HCHO_slant_column_number_density_max,L3_HCHO_HCHO_slant_column_number_density_min,L3_HCHO_HCHO_slant_column_number_density_mean,L3_HCHO_HCHO_slant_column_number_density_std,relative_humidity_2m_above_ground_sum,relative_humidity_2m_above_ground_max,relative_humidity_2m_above_ground_min,relative_humidity_2m_above_ground_mean,relative_humidity_2m_above_ground_std,u_component_of_wind_10m_above_ground_sum,u_component_of_wind_10m_above_ground_max,u_component_of_wind_10m_above_ground_min,u_component_of_wind_10m_above_ground_mean,u_component_of_wind_10m_above_ground_std,precipitable_water_entire_atmosphere_sum,precipitable_water_entire_atmosphere_max,precipitable_water_entire_atmosphere_min,precipitable_water_entire_atmosphere_mean,precipitable_water_entire_atmosphere_std,specific_humidity_2m_above_ground_sum,specific_humidity_2m_above_ground_max,specific_humidity_2m_above_ground_min,specific_humidity_2m_above_ground_mean,specific_humidity_2m_above_ground_std,v_component_of_wind_10m_above_ground_sum,v_component_of_wind_10m_above_ground_max,v_component_of_wind_10m_above_ground_min,v_component_of_wind_10m_above_ground_mean,v_component_of_wind_10m_above_ground_std,temperature_2m_above_ground_sum,temperature_2m_above_ground_max,temperature_2m_above_ground_min,temperature_2m_above_ground_mean,temperature_2m_above_ground_std,pca_AE_sum,pca_AE_max,pca_AE_min,pca_AE_mean,pca_AE_std
0,0OS9LVX X 2020-01-02,2020-01-02,0OS9LVX,11.600000,30.200001,0.00409,14.656824,3.956377,0.712605,0.000053,0.000108,0.466171,0.010752,835670.492740,68.099367,1.445658,-95.984984,22.942019,0.000046,6156.074219,0.000007,0.113310,227.467539,0.032071,68.099367,1.445658,-95.984984,22.942019,0.017953,841.142869,155.982981,835625.785337,32.709708,-95.987015,0.000073,0.032071,0.000088,1.494039,2592.634167,74028.429228,...,0.004408,0.000209,-0.00009,0.000047,0.000048,4133.400084,92.300003,14.900001,43.972341,13.630843,348.839066,6.772317,0.619441,3.711054,1.406078,1477.107926,27.800001,5.569307,15.713914,5.589902,0.603796,0.012407,0.002185,0.006423,0.001924,116.788942,3.873403,-1.605181,1.242436,1.005479,1491.644476,20.240808,11.896295,15.868558,1.671335,279648.204972,3718.49279,2067.325786,2974.980904,472.990428
1,0OS9LVX X 2020-01-03,2020-01-03,0OS9LVX,18.300001,42.900002,0.00595,15.026544,4.230430,0.661892,0.000050,0.000109,-0.213659,0.028307,835281.882757,75.936844,34.638933,-95.017976,18.539524,0.000045,7311.869141,0.000005,0.110397,227.788713,0.044784,75.936845,34.639165,-95.017596,18.539491,0.019576,1187.570320,922.385833,835225.278332,73.808143,-95.017095,0.000048,0.040803,0.000074,1.534485,7334.601102,48466.653300,...,0.004408,0.000209,-0.00009,0.000047,0.000048,4133.400084,92.300003,14.900001,43.972341,13.630843,348.839066,6.772317,0.619441,3.711054,1.

In [0]:
trainag.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
testag.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
target=trainag['target']
trainag.drop('target',axis=1,inplace=True)

In [0]:
feature=['precipitable_water_entire_atmosphere', 	'relative_humidity_2m_above_ground', 	'specific_humidity_2m_above_ground', 	'temperature_2m_above_ground', 	'u_component_of_wind_10m_above_ground', 	'v_component_of_wind_10m_above_ground', 	'L3_NO2_NO2_column_number_density', 'L3_SO2_SO2_column_number_density',	'L3_SO2_SO2_column_number_density_amf' 	,'L3_SO2_SO2_slant_column_number_density', 	 'L3_NO2_tropospheric_NO2_column_number_density', 'L3_HCHO_tropospheric_HCHO_column_number_density', 'L3_CO_CO_column_number_density', 'L3_HCHO_HCHO_slant_column_number_density'	,'L3_CO_H2O_column_number_density'	,'L3_O3_O3_column_number_density'	,'L3_NO2_NO2_slant_column_number_density']
train1=trainag.copy()
test1=testag.copy()
for c in list(feature):
  if c not in l:
    train1.drop(c,inplace=True,axis=1)
    test1.drop(c,inplace=True,axis=1)
    
pca = PCA(random_state=42,n_components=1)
train_pca = pca.fit_transform(train1)
trainag['pca_feature'] = train_pca[:,0]
test_pca = pca.transform(test1)
testag['pca_feature'] = test_pca[:,0]

# modeling

In [0]:

from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
dev_X, val_X, dev_y, val_y = train_test_split(trainag, target, test_size = 0.15, random_state = 42)


In [265]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
    'objective' :'regression',
    'learning_rate' : 0.04,
    'num_iterations': 20000,
    'max_bins': 50, 
    'max_depth' :7 ,
    'num_leaves' : 70,
    'feature_fraction': 0.64, 
    'bagging_fraction': 0.8, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'rmse' ,
     'min_data_in_leaf':5,
    'reg_lambda' :100
     }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 50000, 
                      valid_sets=[lgtrain, lgval], 
                      early_stopping_rounds=400, 
                      verbose_eval=150, 
                      evals_result=evals_result)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result
pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, testag)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 400 rounds.
[150]	training's rmse: 21.0184	valid_1's rmse: 21.5579
[300]	training's rmse: 18.5329	valid_1's rmse: 20.349
[450]	training's rmse: 16.9196	valid_1's rmse: 19.7826
[600]	training's rmse: 15.6247	valid_1's rmse: 19.4358
[750]	training's rmse: 14.5105	valid_1's rmse: 19.1824
[900]	training's rmse: 13.5565	valid_1's rmse: 18.9844
[1050]	training's rmse: 12.6991	valid_1's rmse: 18.8219
[1200]	training's rmse: 11.9375	valid_1's rmse: 18.6939
[1350]	training's rmse: 11.2399	valid_1's rmse: 18.5822
[1500]	training's rmse: 10.5854	valid_1's rmse: 18.4911
[1650]	training's rmse: 9.98892	valid_1's rmse: 18.4253
[1800]	training's rmse: 9.41433	valid_1's rmse: 18.3586
[1950]	training's rmse: 8.87492	valid_1's rmse: 18.298
[2100]	training's rmse: 8.37137	valid_1's rmse: 18.2257
[2250]	training's rmse: 7.89105	valid_1's rmse: 18.1898
[2400]	training's rmse: 7.43301	valid_1's rmse: 18.133
[2550]	training's rmse: 7.00542	valid_1's rmse: 18

In [266]:
!pip install catboost

from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=30000,
                             learning_rate=0.045,
                             depth=8,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=300)
cb_model.fit(dev_X, dev_y,
             eval_set=(val_X, val_y),
             use_best_model=True,
             verbose=50)


0:	learn: 45.9160705	test: 45.1641550	best: 45.1641550 (0)	total: 381ms	remaining: 3h 10m 25s
50:	learn: 27.3851519	test: 26.8203190	best: 26.8203190 (50)	total: 7.61s	remaining: 1h 14m 30s
100:	learn: 24.0073256	test: 24.0628887	best: 24.0628887 (100)	total: 14.7s	remaining: 1h 12m 32s
150:	learn: 22.3139192	test: 22.9218920	best: 22.9218920 (150)	total: 21.7s	remaining: 1h 11m 25s
200:	learn: 21.1337211	test: 22.2527869	best: 22.2527869 (200)	total: 28.6s	remaining: 1h 10m 42s
250:	learn: 20.1439440	test: 21.7074952	best: 21.7074952 (250)	total: 35.6s	remaining: 1h 10m 16s
300:	learn: 19.2088617	test: 21.2104569	best: 21.2104569 (300)	total: 42.6s	remaining: 1h 10m
350:	learn: 18.4168829	test: 20.8332272	best: 20.8332272 (350)	total: 49.5s	remaining: 1h 9m 44s
400:	learn: 17.7309989	test: 20.5325366	best: 20.5325366 (400)	total: 56.5s	remaining: 1h 9m 30s
450:	learn: 17.1422021	test: 20.2854656	best: 20.2854656 (450)	total: 1m 3s	remaining: 1h 9m 17s
500:	learn: 16.5820175	test: 20.0

In [0]:
pred_test_cat2 = (cb_model.predict(testag))


In [0]:
#Ensembling

In [0]:
x=(pred_test_cat2*0.523+pred_test_cat*0.252+0.225*pred_test)

In [0]:
results_df = pd.DataFrame({"Place_ID X Date":sub['Place_ID X Date'], "target": np.clip(x, 0, a_max=None)})

results_df.to_csv("emino.csv", index=False)

In [295]:
results_df.head()

,Place_ID X Date,target
0,0OS9LVX X 2020-01-02,38.930770
1,0OS9LVX X 2020-01-03,32.668566
2,0OS9LVX X 2020-01-04,32.635173
3,0OS9LVX X 2020-01-05,31.799213
4,0OS9LVX X 2020-01-06,30.547980


In [0]:
           #0.2,0.6,0.2